In [302]:
import pandas as pd
import os
import math
import pickle
import configparser
# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [312]:
import json
with open("c:/dev/kolumbus-atl/out/single/monitor_passengers.log") as file:
    txt=file.read()

In [313]:
print(txt[:400])

[{'name': 1, 'bus_stop_id': 3.0, 'platform_nr': 1.0, 'passenger_id': 73, 'id': 1, 'trip_id': 1, 'time': 18172.46843005962, 'type': 'bus', 'action': 'bus_onboard_passenger', 'route': 'A'},
{'from_stop': 3.0, 'to_stop': 48.0, 'start_time': 18139.504843918083, 'boarded_time': 18172.46843005962, 'waited_time': 32.963586141537235, 'bus_id': 1, 'id': 73, 'time': 18172.46843005962, 'type': 'passenger', '


In [314]:
    txt = txt.replace("[","")
    txt = txt.replace("]","")
    txt = txt.replace("'","")
    txt = txt.replace("{","")
    txt = txt.replace("},","")
    rows = txt.split("\n")
    print(rows[0])
    print(rows[1])
    print(rows[2])
    print(rows[-2])
    print(rows[-1])
    rows2 = []
    for idx, row in enumerate(rows):
        lst = [tuple(kv.split(":")) for kv in row.split(", ")]
        lst_dict = {}
        for tpl in lst:
            try:
                lst_dict[tpl[0]] = tpl[1]
            except:
                pass
        rows2.append(lst_dict)
# print(rows2[0])
df = pd.DataFrame(rows2)

name: 1, bus_stop_id: 3.0, platform_nr: 1.0, passenger_id: 73, id: 1, trip_id: 1, time: 18172.46843005962, type: bus, action: bus_onboard_passenger, route: A
from_stop: 3.0, to_stop: 48.0, start_time: 18139.504843918083, boarded_time: 18172.46843005962, waited_time: 32.963586141537235, bus_id: 1, id: 73, time: 18172.46843005962, type: passenger, action: boarded
name: 1, bus_stop_id: 4.0, platform_nr: 1.0, passenger_id: 129, id: 1, trip_id: 1, time: 18261.058048212075, type: bus, action: bus_onboard_passenger, route: A
from_stop: 66.0, to_stop: 41.0, start_time: 18466.95371367025, boarded_time: 34140.962696223185, waited_time: 15674.008982552936, bus_id: 1, id: 242, time: 34140.962696223185, type: passenger, action: boarded



In [315]:
df['boarded_time'] = df['boarded_time'].astype(float)
df['time'] = df['time'].astype(float)
df['start_time'] = df['start_time'].astype(float)
df['waited_time'] = df['waited_time'].astype(float)
df['passenger_id'] = df['passenger_id'].astype(float)
df.head(3)

,action,boarded_time,bus_id,bus_stop_id,from_stop,id,name,passenger_id,platform_nr,route,start_time,time,to_stop,total_time,travelled_time,trip_id,type,waited_time
0,bus_onboard_passenger,NaN,NaN,3.0,NaN,1,1,73.0,1.0,A,NaN,18172.468430,NaN,NaN,NaN,1,bus,NaN
1,boarded,18172.46843,1,NaN,3.0,73,NaN,NaN,NaN,NaN,18139.504844,18172.468430,48.0,NaN,NaN,NaN,passenger,32.963586
2,bus_onboard_passenger,NaN,NaN,4.0,NaN,1,1,129.0,1.0,A,NaN,18261.058048,NaN,NaN,NaN,1,bus,NaN


In [316]:
df = df[df.passenger_id.isna() == False]
df.head(5)

,action,boarded_time,bus_id,bus_stop_id,from_stop,id,name,passenger_id,platform_nr,route,start_time,time,to_stop,total_time,travelled_time,trip_id,type,waited_time
0,bus_onboard_passenger,NaN,NaN,3.0,NaN,1,1,73.0,1.0,A,NaN,18172.468430,NaN,NaN,NaN,1,bus,NaN
2,bus_onboard_passenger,NaN,NaN,4.0,NaN,1,1,129.0,1.0,A,NaN,18261.058048,NaN,NaN,NaN,1,bus,NaN
4,bus_onboard_passenger,NaN,NaN,6.0,NaN,1,1,172.0,0.0,A,NaN,18426.880983,NaN,NaN,NaN,1,bus,NaN
6,bus_onboard_passenger,NaN,NaN,7.0,NaN,1,1,11.0,0.0,A,NaN,18506.280961,NaN,NaN,NaN,1,bus,NaN
8,bus_onboard_passenger,NaN,NaN,8.0,NaN,1,1,238.0,0.0,A,NaN,18581.285416,NaN,NaN,NaN,1,bus,NaN


In [317]:
df[df.passenger_id == 73]

,action,boarded_time,bus_id,bus_stop_id,from_stop,id,name,passenger_id,platform_nr,route,start_time,time,to_stop,total_time,travelled_time,trip_id,type,waited_time
0,bus_onboard_passenger,NaN,NaN,3.0,NaN,1,1,73.0,1.0,A,NaN,18172.468430,NaN,NaN,NaN,1,bus,NaN
985,bus_release_passenger,NaN,NaN,48.0,NaN,1,1,73.0,1.0,A,NaN,21913.139474,NaN,NaN,NaN,1,bus,NaN


In [318]:
# df = df[df.groupby('passenger_id')['time'].diff().isna() == False]

In [319]:
group_max = df.groupby('passenger_id').agg({'time': 'max', 'passenger_id': 'first', 'bus_stop_id': 'first'})
group_min = df.groupby('passenger_id').agg({'time': 'min', 'passenger_id': 'first', 'bus_stop_id': 'first'})
group_max['diff'] = group_max.time - group_min.time
group_max.sort_values(by=['diff'], ascending=False).head(10)

,time,passenger_id,bus_stop_id,diff
passenger_id,,,,
446.0,28950.332540,446.0,65.0,5414.879292
2803.0,28585.277642,2803.0,64.0,4948.290427
116.0,28838.110113,116.0,60.0,4538.520762
790.0,27833.656178,790.0,65.0,4298.202930
1103.0,28082.916110,1103.0,61.0,4148.664282
5593.0,33298.251303,5593.0,16.0,4109.450361
2550.0,33298.251303,2550.0,16.0,4109.450361
459.0,22992.212603,459.0,15.0,3975.769055
1554.0,27833.656178,1554.0,61.0,3899.404350


In [320]:
df[df.passenger_id == 446.0]

,action,boarded_time,bus_id,bus_stop_id,from_stop,id,name,passenger_id,platform_nr,route,start_time,time,to_stop,total_time,travelled_time,trip_id,type,waited_time
1324,bus_onboard_passenger,NaN,NaN,65.0,NaN,1,1,446.0,0.0,A,NaN,23535.453248,NaN,NaN,NaN,1,bus,NaN
2825,bus_release_passenger,NaN,NaN,3.0,NaN,1,1,446.0,0.0,A,NaN,28950.332540,NaN,NaN,NaN,1,bus,NaN
